# ✏️ SOLUÇÕES: Introdução aos principais conceitos de modelos preditivos.

---

## 📋 Índice

1. [Exercício 1: Classificação com Dataset Diabetes](#exercicio1)
2. [Exercício 2: Regressão com Dataset Housing](#exercicio2)

---

Este notebook contém as **soluções completas** para todos os exercícios do notebook principal.

**⚠️ IMPORTANTE:** Tenta resolver os exercícios primeiro antes de consultares as soluções!

In [ ]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================
import pandas as pd
import numpy as np
from google.colab import files
import io
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

print("✅ Todas as bibliotecas importadas!")

In [ ]:
# ============================================================================
# FUNÇÃO DE PRÉ-PROCESSAMENTO
# ============================================================================
def process_data(X, num_vars, cat_vars):
    """
    Processa dados para machine learning: normaliza numéricas e codifica categóricas.
    """
    scaler = StandardScaler()
    enc = OneHotEncoder(sparse_output=False, drop="first")
    
    if len(num_vars) > 0:
        X_num = scaler.fit_transform(X[num_vars])
    else:
        X_num = np.array([]).reshape(X.shape[0], 0)
    
    if len(cat_vars) > 0:
        X_cat = enc.fit_transform(X[cat_vars])
    else:
        X_cat = np.array([]).reshape(X.shape[0], 0)
    
    if X_num.shape[1] > 0 and X_cat.shape[1] > 0:
        X_processed = np.concatenate([X_num, X_cat], axis=1)
    elif X_num.shape[1] > 0:
        X_processed = X_num
    else:
        X_processed = X_cat
    
    return X_processed

print("✅ Função process_data() carregada!")

# <a name="exercicio1"></a> 📊 Exercício 1: Classificação com Dataset Diabetes

## Solução 1.1: Identificação de Variáveis

**Solução:** Identificar variáveis numéricas (exceto Outcome) e categóricas.

In [ ]:
# ============================================================================
# CARREGAMENTO DO DATASET
# ============================================================================
# Opção 1: Upload do ficheiro (descomenta se necessário)
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))

# Opção 2: Se o ficheiro já estiver no Colab, utiliza:
# df = pd.read_csv('diabetes.csv')

# Verificação
print("✅ Dataset carregado!")
print(f"   Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
print(f"   Colunas: {list(df.columns)}")
print(f"\nPrimeiras linhas:")
print(df.head())

In [ ]:
# ============================================================================
# SOLUÇÃO 1.1: IDENTIFICAÇÃO DE VARIÁVEIS
# ============================================================================
# Identificar tipos numéricos
numerics = ["int64", "float64"]

# Selecionar variáveis numéricas e remover "Outcome"
num_vars = df.select_dtypes(include=numerics).columns.difference(["Outcome"]).tolist()

# Selecionar variáveis categóricas (object)
cat_vars = df.select_dtypes(include="object").columns.tolist()

# Verificação
print("Variáveis identificadas:")
print(f"  Numéricas ({len(num_vars)}): {num_vars}")
print(f"  Categóricas ({len(cat_vars)}): {cat_vars}")
print(f"  Target: Outcome")

# Validação
assert "Outcome" not in num_vars, "❌ ERRO: Outcome não deve estar em num_vars!"
assert "Outcome" not in cat_vars, "❌ ERRO: Outcome não deve estar em cat_vars!"
assert len(num_vars) > 0, "❌ ERRO: Deve haver pelo menos uma variável numérica!"
print("\n✅ Estrutura de variáveis correta!")

## Solução 1.2: Pré-processamento

**Solução:** Separar X e y, e processar os dados usando a função `process_data()`.

In [ ]:
# ============================================================================
# SOLUÇÃO 1.2: PRÉ-PROCESSAMENTO
# ============================================================================
# Separar variáveis independentes (X) e target (y)
X = df.drop("Outcome", axis=1)  # Todas as colunas exceto "Outcome"
y = df["Outcome"]  # Apenas a coluna "Outcome"

# Processar os dados usando a função process_data
X_processed = process_data(X, num_vars, cat_vars)

print("Pré-processamento concluído!")
print(f"  X.shape: {X.shape}")
print(f"  y.shape: {y.shape}")
print(f"  X_processed.shape: {X_processed.shape}")
print(f"  Tipo de X_processed: {type(X_processed)}")

# Validação
assert X_processed.shape[0] == len(df), "❌ ERRO: Número de linhas não coincide!"
assert isinstance(X_processed, np.ndarray), "❌ ERRO: X_processed deve ser um array numpy!"
assert "Outcome" not in X.columns, "❌ ERRO: Outcome não deve estar em X!"
print("\n✅ Pré-processamento correto!")

## Solução 1.3: Divisão Treino/Teste

**Solução:** Dividir o dataset em 80% de treino e 20% de teste usando `train_test_split()`.

In [ ]:
# ============================================================================
# SOLUÇÃO 1.3: DIVISÃO TREINO/TESTE
# ============================================================================
# Dividir em 80% treino e 20% teste
# IMPORTANTE: Usar X_processed (dados processados) e não X original!
X_train, X_test, y_train, y_test = train_test_split(
    X_processed,  # Usar dados processados!
    y, 
    test_size=0.2,  # 20% para teste = 80% para treino
    random_state=42  # Para reprodutibilidade
)

print(f"✅ Divisão concluída!")
print(f"  Treino: {X_train.shape[0]} amostras")
print(f"  Teste: {X_test.shape[0]} amostras")

## Solução 1.4: Treino e Avaliação KNN

**Solução:** Treinar o modelo KNN e avaliar sua performance com matriz de confusão e classification report.

In [ ]:
# ============================================================================
# SOLUÇÃO 1.4: TREINO E AVALIAÇÃO KNN
# ============================================================================
# Criar e treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)  # Usar 5 vizinhos (valor comum)
knn.fit(X_train, y_train)  # Treinar com dados de treino

# Fazer previsões nos dados de teste
y_pred = knn.predict(X_test)

# Criar e visualizar matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=df["Outcome"].unique())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=df["Outcome"].unique())
disp.plot()
plt.title("Matriz de Confusão - KNN")
plt.show()

# Mostrar classification report (accuracy, precision, recall, f1-score)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))

# Calcular e mostrar accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# <a name="exercicio2"></a> 🏠 Exercício 2: Regressão com Dataset Housing

## Soluções das Funções

**Solução:** Completar todas as funções do pipeline de regressão.

In [ ]:
# ============================================================================
# SOLUÇÃO: FUNÇÕES DO PIPELINE DE REGRESSÃO
# ============================================================================

def process_data(X, num_vars, cat_vars):
    """
    Processa dados para machine learning: normaliza numéricas e codifica categóricas.
    """
    scaler = StandardScaler()
    enc = OneHotEncoder(sparse_output=False, drop="first")
    
    if len(num_vars) > 0:
        X_num = scaler.fit_transform(X[num_vars])
    else:
        X_num = np.array([]).reshape(X.shape[0], 0)
    
    if len(cat_vars) > 0:
        X_cat = enc.fit_transform(X[cat_vars])
    else:
        X_cat = np.array([]).reshape(X.shape[0], 0)
    
    if X_num.shape[1] > 0 and X_cat.shape[1] > 0:
        X_processed = np.concatenate([X_num, X_cat], axis=1)
    elif X_num.shape[1] > 0:
        X_processed = X_num
    else:
        X_processed = X_cat
    
    return X_processed


def split_data(X, y, test_portion):
    """
    Divide os dados em treino e teste.
    """
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_portion, 
        random_state=42
    )
    return X_train, X_test, y_train, y_test


def train_model(X_train, X_test, y_train):
    """
    Treina um modelo de regressão linear e faz previsões.
    """
    reg = LinearRegression().fit(X_train, y_train)  # Fazer o fitting aos dados de treino
    y_pred = reg.predict(X_test)
    return y_pred


def get_error(y_pred, y_true):
    """
    Calcula o erro e visualiza previsões vs valores reais.
    """
    print(f"Erro absoluto médio = {mean_absolute_error(y_true, y_pred)}")
    
    plt.figure(figsize=(12, 6))
    plt.plot(range(0, y_true.shape[0]), y_true, color="blue", label="Valores reais", linewidth=1.5)
    plt.plot(range(0, y_pred.shape[0]), y_pred, color="red", label="Valores previstos", linewidth=0.5)
    plt.xlabel("Amostra")
    plt.ylabel("Preço")
    plt.title("Comparação: Valores Reais vs Previstos")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

print("✅ Todas as funções criadas com sucesso!")

## Solução: Pipeline Completo

**Solução:** Identificar o target, processar dados, dividir treino/teste, treinar modelo e calcular métricas.

In [ ]:
# Carregar dataset housing
uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['housing.csv']))

print("✅ Dataset housing carregado!")
print(f"   Dimensões: {df2.shape[0]} linhas × {df2.shape[1]} colunas")
print(f"   Colunas: {list(df2.columns)}")
print(f"\nPrimeiras linhas:")
print(df2.head())
print(f"\nTipos de dados:")
print(df2.dtypes)

In [ ]:
# ============================================================================
# SOLUÇÃO: PIPELINE COMPLETO DE REGRESSÃO
# ============================================================================
target = "MEDV"  # OU "median_house_value" dependendo do dataset

# Identificar as variáveis explicativas/independentes
independent_vars = df2.columns.difference([target])

# Dividir o target das variáveis explicativas
X, y = df2[independent_vars], df2[target]

# Identificar as variáveis categóricas
cat_vars = df2[independent_vars].select_dtypes(include="object").columns.tolist()

# Identificar as variáveis numéricas (todas menos as categóricas e o target)
num_vars = df2[independent_vars].select_dtypes(include=["int64", "float64"]).columns.tolist()

print(f"\n📊 Variáveis identificadas:")
print(f"  Numéricas ({len(num_vars)}): {num_vars}")
print(f"  Categóricas ({len(cat_vars)}): {cat_vars}")
print(f"  Target: {target}")

# Processar os dados (normalização e one-hot encoding)
X_processed = process_data(X, num_vars, cat_vars)

# Dividir o dataset em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = split_data(X_processed, y, 0.3)

# Treinar o modelo e fazer previsões
y_pred = train_model(X_train, X_test, y_train)

# Calcular e visualizar erros
get_error(y_pred, y_test)

# Calcular e mostrar R² (coeficiente de determinação)
r2 = r2_score(y_test, y_pred)
print(f"\n✅ R² (Coeficiente de Determinação) = {r2:.4f}")
print(f"   (R² próximo de 1 indica bom ajuste do modelo)")